<h3 style="text-align: center; font-family: Arial, sans-serif; color: #4CAF50;">TSS - Temporal Similarity Search</h3>
<ul style="font-family: Arial, sans-serif; font-size: 12pt; color: #333;">
</ul>


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Charger les données
df = pd.read_csv("../Datasources/MetroPT3_imputed_final.csv", delimiter=",", decimal=".", index_col=0)
df.reset_index(drop=True, inplace=True)

# Convertir la colonne timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# Sélection des colonnes continues et catégoriques
continuous_features = ["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"]
categorical_features = ["COMP", "DV_eletric", "Towers", "LPS", "Pressure_switch", "Oil_level", "Caudal_impulses"]
columns_to_keep = ["timestamp", "panne"] + continuous_features + categorical_features

# Filtrer les colonnes utiles
df = df[columns_to_keep]


In [ ]:
display(df.head(3))

In [2]:
# Supprimer la colonne 'panne' de df
df = df.drop(columns=['panne'])

# Vérifier si la colonne a été supprimée
display(df.head())


,timestamp,TP2,DV_pressure,Oil_temperature,Motor_current,Reservoirs,COMP,DV_eletric,Towers,LPS,Pressure_switch,Oil_level,Caudal_impulses
0,2020-02-01 00:00:00,-0.012,-0.024,53.600,0.0400,9.358,1.0,0.0,1.0,0.0,1.0,1.0,1.0
1,2020-02-01 00:00:10,-0.014,-0.022,53.675,0.0400,9.348,1.0,0.0,1.0,0.0,1.0,1.0,1.0
2,2020-02-01 00:00:20,-0.012,-0.022,53.600,0.0425,9.338,1.0,0.0,1.0,0.0,1.0,1.0,1.0
3,2020-02-01 00:00:30,-0.012,-0.022,53.425,0.0400,9.328,1.0,0.0,1.0,0.0,1.0,1.0,1.0
4,2020-02-01 00:00:40,-0.012,-0.022,53.475,0.0400,9.318,1.0,0.0,1.0,0.0,1.0,1.0,1.0


In [3]:
# Ajouter les classes de panne
# Classe 0 : Pas de panne détectée
# Classe 1 : En pleine panne
# Classe 2 : Panne prévue dans moins de 30 minutes

# Liste des intervalles de pannes
pannes = [
    {'start': '2020-04-18 00:00:00', 'end': '2020-04-18 23:59:00'},
    {'start': '2020-05-29 23:30:00', 'end': '2020-05-30 06:00:00'},
    {'start': '2020-06-05 10:00:00', 'end': '2020-06-07 14:30:00'},
    {'start': '2020-07-15 14:30:00', 'end': '2020-07-15 19:00:00'},
         ]

# Convertir les timestamps des pannes en datetime
for panne in pannes:
    panne['start'] = pd.to_datetime(panne['start'])
    panne['end']   = pd.to_datetime(panne['end'])

# Ajouter une colonne 'panne' avec la valeur par défaut 0 (aucune panne détectée)
df['panne'] = 0


for panne in pannes:
    # Classe 1 : En pleine panne
    df.loc[(df['timestamp'] >= panne['start']) & (df['timestamp'] <= panne['end']), 'panne'] = 1
    # Classe 2 : Panne prévue dans moins de 15 minutes
    df.loc[(df['timestamp'] < panne['start']) & (df['timestamp'] >= panne['start'] - pd.Timedelta(minutes=15)), 'panne'] = 2

# Afficher un aperçu du DataFrame avec la nouvelle colonne 'panne'
display(df.head(50))

,timestamp,TP2,DV_pressure,Oil_temperature,Motor_current,Reservoirs,COMP,DV_eletric,Towers,LPS,Pressure_switch,Oil_level,Caudal_impulses,panne
0,2020-02-01 00:00:00,-0.012,-0.024,53.600,0.0400,9.358,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
1,2020-02-01 00:00:10,-0.014,-0.022,53.675,0.0400,9.348,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
2,2020-02-01 00:00:20,-0.012,-0.022,53.600,0.0425,9.338,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
3,2020-02-01 00:00:30,-0.012,-0.022,53.425,0.0400,9.328,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
4,2020-02-01 00:00:40,-0.012,-0.022,53.475,0.0400,9.318,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
5,2020-02-01 00:00:50,-0.012,-0.024,53.500,0.0400,9.308,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
6,2020-02-01 00:01:00,-0.012,-0.024,53.375,0.0400,9.298,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
7,2020-02-01 00:01:10,-0.014,-0.024,53.550,0.0400,9.286,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
8,2020-02-01 00:01:20,-0.012,-0.022,53.425,0.0400,9.276,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
9,2020-02-01 00:01:30,-0.012,-0.022,53.375,0.0400,9.264,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0


In [4]:
###################################################################################################
################ Train : Panne1   (1J ET DEMI)                                              #######
################ Test  : Panne4  (15mn + 15mn + 15mn)     #########################################
###################################################################################################

# Classe 0 : Pas de panne détectée
# Classe 1 : En pleine panne
# Classe 2 : Panne prévue dans moins de 30 minutes

pannes = [
    {'start': '2020-04-18 00:00:00', 'end': '2020-04-18 23:59:00'},
    {'start': '2020-05-29 23:30:00', 'end': '2020-05-30 06:00:00'},
    {'start': '2020-06-05 10:00:00', 'end': '2020-06-07 14:30:00'},
    {'start': '2020-07-15 14:30:00', 'end': '2020-07-15 19:00:00'},
         ]


# Définir les périodes d'entraînement et de test
train_periods = [{'start': '2020-02-02 00:00', 'end': '2020-06-07 14:30:00'}]
test_periods  = [{'start': '2020-06-07 14:30:10', 'end': '2020-07-15 19:00:00'}]

# Séparer les données pour l'entraînement
train_df = pd.concat([
    df[(df['timestamp'] >= pd.Timestamp(period['start'])) & 
       (df['timestamp'] <= pd.Timestamp(period['end']))]
    for period in train_periods
])

# Séparer les données pour le test
test_df = pd.concat([
    df[(df['timestamp'] >= pd.Timestamp(period['start'])) & 
       (df['timestamp'] <= pd.Timestamp(period['end']))]
    for period in test_periods
])

print(f"Entrainement : {len(train_df)} lignes")
print(f"Test : {len(test_df)} lignes")

Entrainement : 1093861 lignes
Test : 329940 lignes


In [5]:
# Distribution des modalités de la variable 'panne' dans l'ensemble d'entraînement
print("Modalités de 'panne' dans l'ensemble d'entraînement :")
print(train_df["panne"].value_counts())

# Distribution des modalités de la variable 'panne' dans l'ensemble de test
print("\nModalités de 'panne' dans l'ensemble de test :")
print(test_df["panne"].value_counts())


Modalités de 'panne' dans l'ensemble d'entraînement :
0    1063714
1      29877
2        270
Name: panne, dtype: int64

Modalités de 'panne' dans l'ensemble de test :
0    328229
1      1621
2        90
Name: panne, dtype: int64


In [6]:
# Paramètres des fenêtres
window_size = 100  # Taille de la fenêtre
step_size = 1     # Pas de glissement
numeric_columns = continuous_features

# Générer les fenêtres pour l'entraînement
train_windows = []
for i in tqdm(range(0, len(train_df) - window_size + 1, step_size)):
    window_data = train_df.iloc[i:i + window_size][numeric_columns].values.flatten()
    panne_value = train_df.iloc[i + window_size - 1]["panne"]  # Dernière valeur de panne dans la fenêtre
    train_windows.append({
        "index": i,
        "timestamp": train_df.iloc[i]["timestamp"],
        "sensor_data": window_data.tolist(),
        "panne": panne_value
    })

# Générer les fenêtres pour le test
test_windows = []
for i in tqdm(range(0, len(test_df) - window_size + 1, step_size)):
    window_data = test_df.iloc[i:i + window_size][numeric_columns].values.flatten()
    panne_value = test_df.iloc[i + window_size - 1]["panne"]  # Dernière valeur de panne dans la fenêtre
    test_windows.append({
        "index": i,
        "timestamp": test_df.iloc[i]["timestamp"],
        "sensor_data": window_data.tolist(),
        "panne": panne_value
    })

print(f"Fenêtres d'entraînement : {len(train_windows)}")
print(f"Fenêtres de test : {len(test_windows)}")


100%|█████████████████████████████████████████████████████████████████████████| 329841/329841 [20:40<00:00, 265.94it/s]

Fenêtres d'entraînement : 1093762
Fenêtres de test : 329841


In [ ]:
import kdbai_client as kdbai

# Connexion à KDB.AI
session = kdbai.Session(endpoint="https://cloud.kdb.ai/instance/atr0v2owym", api_key="a5b897a7b4-Av/lT//S3W++g8K11vVKfJo5rIj5gK492sbpts/1f+cr/5m5n5JluPcqcQR2gGiiQQ7SrCFAc7ma2RhT")
db = session.database('default')



In [ ]:

# Schéma de la table
schema = [
    {"name": "index", "type": "int64"},
    {"name": "timestamp", "type": "datetime64[ns]"},
    {"name": "sensor_data", "type": "float64s"},
    {"name": "panne", "type": "int64"}
]
indexes = [
    {"name": "flat_index", "type": "flat", "column": "sensor_data", "params": {"metric": "L2","dims": 10}}
]

# Création de la table
train_table = db.create_table("train_sensors", schema=schema, indexes=indexes)

# Insertion des données d'entraînement
batch_size = 100
for i in tqdm(range(0, len(train_windows), batch_size)):
    batch = train_windows[i:i + batch_size]
    train_table.insert(batch)

In [ ]:
# Parcourir toutes les fenêtres de test
for i, test_window in enumerate(test_windows):
    # Récupérer le vecteur de test
    query_vector = test_window["sensor_data"]

    # Recherche vectorielle
    results = train_table.search(
        vectors={"flat_index": [query_vector]},
        n=5  # Trouver les 5 motifs les plus similaires
    )

    # Extraire les valeurs de panne des motifs similaires
    similar_pannes = results[0]["panne"].tolist()

    # Prédiction par majorité
    predicted_panne = max(set(similar_pannes), key=similar_pannes.count)

    # Affichage simple
    print(f"Fenêtre {i} - Prédiction : {predicted_panne} - Valeur de Test : {test_window['panne']}")


In [ ]:
# Indice de la fenêtre 11 (les indices Python commencent à 0)
window_index = 10   # Fenêtre 622 correspond à l'index 621

# Récupérer les informations de la fenêtre
window_11 = test_windows[window_index]

# Affichage des détails
print(f"Fenêtre 11 : {window_11}")

In [ ]:
##################################
###### SUPPRIMER LES TABLES ######
##################################
# Accéder aux tables existantes
train_table = db.table("train_sensors")

# Supprimer mes 2 tables
try:
    train_table.drop()
    print("Table 'train_sensors' supprimée avec succès.")
except Exception as e:
    print(f"Erreur lors de la suppression de 'train_sensors' : {e}")

In [ ]:
# Compter les fenêtres dans l'ensemble d'entraînement
num_train_windows = len(train_windows)

# Compter les fenêtres dans l'ensemble de test
num_test_windows = len(test_windows)

# Affichage
print(f"Nombre de fenêtres dans l'ensemble d'entraînement : {num_train_windows}")
print(f"Nombre de fenêtres dans l'ensemble de test : {num_test_windows}")
